In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\m2003004\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\m2003004\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\m2003004\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_dataset = pd.read_json("./data.json").T
train_dataset

,file_id,file_class,file_body
0,5908cb5da047d6c9e6dfea6337fb3189.doc,Договоры для акселератора/Договоры поставки,Evaluation Only. Created with Aspose.Words. Co...
1,14711e4fc8e56f0c75856c8837ec04cb.doc,Договоры для акселератора/Договоры поставки,Evaluation Only. Created with Aspose.Words. Co...
2,7eb67b5aecf3f3190aab0a5f8ea32172.docx,Договоры для акселератора/Договоры поставки,Evaluation Only. Created with Aspose.Words. Co...
3,b40a9d048b199d5f4db62a6a2335f2a0.pdf,Договоры поставки,\n \nДОГОВОР ПОСТАВКИ № 1 \nг. Москва ...
4,84fec112d02288861e7af59f468131fb.docx,Договоры для акселератора/Договоры поставки,Evaluation Only. Created with Aspose.Words. Co...
...,...,...,...
115,f57fe87f15a6dee2b17e804421be63b5.pdf,Договоры купли-продажи,"\nДОГОВОР КУПЛИ-ПРОДАЖИ ………. \nОБОРУДОВАНИЯ, ..."
116,1ea8809d696a4bd6a2076fbc6fd28c23.doc,Договоры для акселератора/Договоры купли-продажи,Evaluation Only. Created with Aspose.Words. Co...
117,57962abd8dbe0ce0c7056896ee4501f1.doc,Договоры для акселератора/Договоры купли-продажи,Evaluation Only. Created with Aspose.Words. Co...
118,2c81df29db63aebf495106881a52188f.doc,Договоры для акселератора/Договоры купли-продажи,Evaluation Only. Created with Aspose.Words. Co...


In [3]:
def text_class_preprocess(class_name):
    class_name = class_name.replace("Договоры для акселератора/", "")
    return class_name

In [4]:
train_dataset["file_class"] = train_dataset["file_class"].apply(text_class_preprocess)
class_code = {"Договоры оказания услуг": 1, "Договоры аренды": 2, "Договоры купли-продажи": 3, "Договоры поставки": 4, "Договоры подряда": 5}
train_dataset["file_class"] = train_dataset["file_class"].map(class_code)

In [5]:
def text_preprocess(text):
    text = text.replace("Evaluation Only. Created with Aspose.Words. Copyright 2003-2022 Aspose Pty Ltd.", "")
    text = text.replace("Created with an evaluation copy of Aspose.Words. To discover the full versions of our APIs please visit: https://products.aspose.com/words/", "")
    text = text.replace('\t', ' ')
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace(u'\xa0', u' ')
    text = text.replace(u'\x07', u' ')
    text = text.replace(u'\x13', u' ')
    text = text.replace(u'\x14', u' ')
    text = text.replace(u'\x15', u' ')
    regular = r'[\*+\#+\№\"\-+\+\=+\?+\_+\«+\»+\!+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
    text = re.sub(regular, '', text)
    text = re.sub(r'(\d+\s\d+)|(\d+)','', text)
    return text

In [6]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-…]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    doc_text = ' '.join([token for token in tokens])
    return doc_text

In [7]:
train_dataset["file_body"] = train_dataset["file_body"].apply(text_preprocess)
train_dataset["file_body"] = train_dataset["file_body"].apply(lemmatize)

In [8]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=text_preprocess)
tfidf = tfidf_vectorizer.fit_transform(train_dataset.file_body.values)

In [9]:
data = tfidf.todense().tolist()
names = tfidf_vectorizer.get_feature_names_out()
word_df = pd.DataFrame(data, columns = names)
word_df['продавец']

0      0.000000
1      0.014869
2      0.000000
3      0.000000
4      0.000000
         ...   
115    0.223354
116    0.280716
117    0.325405
118    0.293483
119    0.388822
Name: продавец, Length: 120, dtype: float64

In [10]:
word_df

,аа,абв,абз,абзац,абонементный,абонент,абонентский,ава,аванс,авансирование,...,ямало,ямалоненецкий,ями,январь,ясный,яха,ячейка,ящик,ёмка,ёмкость
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022309,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.086147,0.0,0.0,0.0
117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.076976,0.0,0.0,0.0
118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [11]:
X_train, X_test, y_train, y_test = train_test_split(word_df, train_dataset['file_class'], random_state=0, test_size = 0.2)

In [12]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [13]:
y_pred =clf.predict(X_test)

In [14]:
clf.predict_proba(X_test)

array([[0.63912755, 0.0734641 , 0.08339367, 0.06335856, 0.14065612],
       [0.09639865, 0.65807168, 0.10893047, 0.06620997, 0.07038924],
       [0.07230121, 0.76559875, 0.06334504, 0.05044949, 0.04830551],
       [0.13505643, 0.07547215, 0.17683998, 0.51221152, 0.10041993],
       [0.11006063, 0.18854997, 0.50617535, 0.11294646, 0.0822676 ],
       [0.69096632, 0.07095701, 0.07904444, 0.05518237, 0.10384986],
       [0.10108755, 0.0885654 , 0.13164981, 0.61034236, 0.06835488],
       [0.08612323, 0.08266589, 0.21015412, 0.54502799, 0.07602877],
       [0.68495218, 0.08511905, 0.08420402, 0.06186754, 0.0838572 ],
       [0.06737808, 0.77310351, 0.06323056, 0.0494873 , 0.04680056],
       [0.80655234, 0.04261123, 0.04282711, 0.03827938, 0.06972995],
       [0.59728367, 0.08713841, 0.11192259, 0.069216  , 0.13443933],
       [0.08279928, 0.07802215, 0.11311766, 0.66240537, 0.06365553],
       [0.58306353, 0.08458743, 0.09353122, 0.07185131, 0.16696651],
       [0.10617597, 0.13558221, 0.

In [15]:
clf.score(X_test, y_test)

0.9583333333333334

In [16]:
a = clf.predict_proba(X_test)
probabilities = [i for i in a]

In [17]:
probabilities_1 = [i[0] for i in probabilities]
probabilities_2 = [i[1] for i in probabilities]
probabilities_3 = [i[2] for i in probabilities]
probabilities_4 = [i[3] for i in probabilities]
probabilities_5 = [i[4] for i in probabilities]

In [18]:
answer = {
    'id':train_dataset.iloc[X_test.index]['file_id'],
    'Вероятность Договоры аренды' : probabilities_1,
    'Вероястность Договоры купли-продажи' : probabilities_2,
    'Вероястность Договоры оказания услуг' : probabilities_3,
    'Вероястность Договоры подряда' : probabilities_4,
    'Вероястность Договоры поставки' : probabilities_5    
}

In [19]:
answer

{'id': 48      f8eabec67b770c384d84f9feb7e02543.doc
 94      f3734851311f2a7ac6c66ad7ccbe1d2c.doc
 95      99a11e3e403b7c8b0773108a84839517.doc
 8       214d620d9c54bc83111277dd872d3cb2.pdf
 97      15e3d16cf68646b3383d386aae2a6849.doc
 22      f28d4a853be12515dae73a5912bc5b41.doc
 7       bec0aa38d1383172690a18d16b07f154.doc
 10      2fd747f38e30ae7ce1c9d6e3b907ac5d.doc
 45      82b672721c50b8897a3cf7c98c0d8e0c.rtf
 89      7e0cf1019baa9e1cdabcfc06d522d424.doc
 33     259b481366842141111305897cce7486.docx
 50      02682d726b725f95b9ee85f751c043d0.doc
 2      7eb67b5aecf3f3190aab0a5f8ea32172.docx
 60      453b2cb6e4ba7b8d1a8382773880d313.doc
 119     0ca2f9faecdbc67d6686a9f5b6636eba.doc
 74      a40a0347f0a91f2537ccc67c46d883b7.doc
 30     317e4552102e82bd9e17ba1205cd4367.docx
 43      b95bbf14eb26ac912b1339e321cbca34.doc
 111    a543a2a59bd604ad1224040b14844b49.docx
 76      ef140e96384a29057445d0a889f1ad50.doc
 63      66ee9b692a6b9c690564c92121a76b15.doc
 59      6babd660f1a9192017a

In [38]:
from keybert import KeyBERT

doc = train_dataset[(train_dataset.file_class == 2) | (train_dataset.file_class == 5)].file_body.values
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2))
print(keywords)

[[('договор односторонний', 0.4887), ('договор нижеследующий', 0.4885), ('настоящий договор', 0.4824), ('заключить настоящий', 0.4717), ('расторжение настоящий', 0.4713)], [('иной юрисдикция', 0.5283), ('договор односторонний', 0.5138), ('ооо юридический', 0.5067), ('российский федерация', 0.4896), ('иметь юридический', 0.4887)], [('наименование организация', 0.4979), ('документация проектировщик', 0.4843), ('организация фио', 0.4774), ('страховой организация', 0.4691), ('строительство наименование', 0.4574)], [('договор односторонний', 0.5203), ('расследование односторонний', 0.4999), ('документация провести', 0.4752), ('корреспонденция относительно', 0.457), ('исполнение договор', 0.4566)], [('помещение остановка', 0.4768), ('законодательство коммерческий', 0.466), ('ограничение относительно', 0.4589), ('информация основание', 0.4565), ('исполнитель командировка', 0.4523)], [('продавец настоящий', 0.4541), ('нижеследующий продавец', 0.4504), ('настоящий договор', 0.4387), ('договор н